# Gene mapping

![](./images/Module1/Gene_ID_Conversion.png)
*(Source: david.ncifcrf.gov)*

## Learning Objectives:
1. Understanding different probe set ID.
2. Performing mapping of various protein/gene IDs into gene identifiers and symbols.

### Identifying probe set ID

Gene set or pathway analysis requires that gene sets and expression data use the same type of gene ID (Entrez Gene ID, Gene symbol or probe set ID etc). However, this is frequently not true for the data we have. For example, our gene sets mostly use Entrez Gene ID, but microarray datasets are frequently labeled by probe set ID (or RefSeq transcript ID etc). Therefore, we need to convert or map the probe set IDs to Entrez gene ID. Here, we we will use `GSE48350` dataset that we have used in the previous section for demonstration of gene ID mapping. From GEO `GSE48350` record page or the CEL data file, we know that it uses Affymetrix Human Genome U133 Plus 2.0 Array. We need to use the corresponding <a href="https://bioconductor.org/">Bioconductor</a> annotation package, `hgu133plus2.db`. We can install the package using following R command:

In [1]:
#if (!require("BiocManager", quietly = TRUE)
#   install.packages("BiocManager")
BiocManager::install("hgu133plus2.db", update = F)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org




Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.3 (2022-03-10)



Warning message:
"package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'hgu133plus2.db'"


## Data normalization
In this section, we will inspect the data to discover what scale the data are presented in. The methods we will use later in this training series assume the data are on a `log2` scale; typically in the range of `0` to `16`.

The `exprs` function can retrieve the expression values as a data frame; with one column per-sample and one row per-gene. To view the expression value of the first 5 samples, we can use the following command:
